<a target="_blank" href="https://colab.research.google.com/github/jackma-00/house-price-prediction/blob/main/experiments/1_house_price_feature_backfill.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# <span style="font-width:bold; font-size: 3rem; color:#333;">Feature Backfill<span>

Installing required packages

In [1]:
!pip install hopsworks[python]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.3/640.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Imports

In [1]:
import pandas as pd
import hopsworks
import os

import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv

e:\Learning\house-price-prediction\housepredenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### <span style='color:#ff5f27'> Connect to Hopsworks Feature Store

In [4]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
load_dotenv()

os.environ["HOPSWORKS_API_KEY"]


'gDvedjVbjxyLtag0.79Zt3ihFkMl2s5e62kZQmOR47WHuT36x3BpdmLf3PalB0265CaXFwcxWUBZPnk37'

In [7]:
hopsworks.get_current_project()

Project('TestProj1', 'maheshkumar201095@gmail.com', 'Default project')

In [6]:
proj = hopsworks.login(api_key_value='gDvedjVbjxyLtag0.79Zt3ihFkMl2s5e62kZQmOR47WHuT36x3BpdmLf3PalB0265CaXFwcxWUBZPnk37')

2025-04-15 20:47:13,418 INFO: Initializing external client
2025-04-15 20:47:13,419 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-15 20:47:18,302 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1222838


## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [16]:
# Load properties CSV into Dataframe
properties_csv = r'E:\Learning\house-price-prediction\src\data_scrapper\realtor-data.csv'
properties_df = pd.read_csv(properties_csv)

In [17]:
properties_df

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


## <span style="color:#ff5f27;"> Data Cleaning

In [18]:
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


Drop null values

In [19]:
properties_df.dropna(inplace=True)

Convert column names to lowercase

In [13]:
# Convert column names to lowercase
properties_df.columns = properties_df.columns.str.lower()

**Evaluating candidate features for properties fg**

Primary key:
* id

Timestamp:
* scraping_date

Numerical features:
* agencyid
* bedroomsnumber
* buildingyear
* codcom
* gsm
* surface
* latitude
* longitude

Boolean features:
* isluxury
* isnew
* on_the_market
* zeroenergybuilding

Categorical features:
* airconditioning
* bathrooms
* city
* condition
* energyclass
* ga4heating
* garage
* heatingtype
* pricerange
* rooms
* id_zona_omi

Label:
* price

In [20]:
# properties selected columns
properties_df = properties_df[['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size','prev_sold_date' ]]

In [21]:
properties_df['prev_sold_date'] = pd.to_datetime(properties_df['prev_sold_date'])
# properties_df = properties_df.sort_values(by=['scraping_date'])

In [27]:
properties_df.reset_index(inplace=True)

In [28]:
properties_df

,index,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,502,92147.0,for_sale,110000.0,7.0,3.0,0.09,1842706.0,Dorado,Puerto Rico,949.0,1192.0,2019-06-28
1,2270,94933.0,for_sale,950000.0,5.0,4.0,0.99,1260473.0,Saint Thomas,Virgin Islands,802.0,5000.0,2013-10-11
2,2277,103341.0,for_sale,6899000.0,4.0,6.0,0.83,17467.0,Saint Thomas,Virgin Islands,802.0,4600.0,2018-04-05
3,3409,21163.0,for_sale,525000.0,3.0,3.0,0.45,1813270.0,Agawam,Massachusetts,1001.0,2314.0,2014-06-25
4,3410,67455.0,for_sale,289900.0,3.0,2.0,0.36,1698080.0,Agawam,Massachusetts,1001.0,1276.0,2012-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084904,2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
1084905,2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
1084906,2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
1084907,2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

In [23]:
fs = proj.get_feature_store()

In [24]:
fs

In [29]:
# Get or create the 'properties' feature group
properties_fg = fs.get_or_create_feature_group(
    name='properties',
    version=4,                                                 # Breaking version
    description='Property Features and Corresponding Prices',
    online_enabled=True,                                       # Necessary for online inference
    primary_key=['index'],
    event_time='prev_sold_date',
)

In [30]:
# Insert data into feature group
properties_fg.insert(properties_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1222838/fs/1209447/fg/1435249


Uploading Dataframe: 100.00% |██████████| Rows 1084909/1084909 | Elapsed Time: 00:52 | Remaining Time: 00:00


Launching job: properties_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1222838/jobs/named/properties_4_offline_fg_materialization/executions


(Job('properties_4_offline_fg_materialization', 'SPARK'), None)

In [31]:
properties_df.to_csv("E:\Learning\house-price-prediction\src\data_scrapper\properties.csv")

In [ ]:
omi_fg = fs.get_or_create_feature_group(
    name='omi',
    version=2,
    description='OMI information',
    online_enabled=True,
    primary_key=['id_zona_omi'],
)

In [ ]:
# Insert data into feature group
omi_fg.insert(omi_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1392972


Uploading Dataframe: 100.00% |██████████| Rows 240570/240570 | Elapsed Time: 00:14 | Remaining Time: 00:00


Launching job: omi_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/omi_2_offline_fg_materialization/executions


(Job('omi_2_offline_fg_materialization', 'SPARK'), None)